In [1]:
# @tool 데코레이터로 랭체인에 함수 연결하기
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='저는 항상 잘 지내고 있습니다! 당신은 어떻게 지내고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 12, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BonlNSfLvGlLBAyac44rQXE7U553f', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ebe6e312-01d4-4fa6-ab7f-fabcdf643465-0', usage_metadata={'input_tokens': 12, 'output_tokens': 20, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
# get_current_time 함수를 랭체인으로 언어 모델에 연결
# 함수에 """ """ 로 표시된 설명은 단순한 주석이 아니라 랭체인에 이 함수의 기능과 입력값, 사용 방법을 알려주는 문서화 문자열(docsstring)입니다.
# 펑션 콜링에서는 딕셔너리로 정보를 작성했지만 이제 문서화 문자열로 작성합니다.

from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time

In [3]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [4]:
# 사용자 질문과 도구를 사용 언어 모델 답변 생성
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1QMzd3XTnx5km4AsvKD4q1yl', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BonlPfp8cyUj1LozLNaDBlrbJf8sJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--04985ec6-dc19-4eeb-8395-06cfac573bd0-0', tool_calls

In [5]:
# 여러 도시를 물을 경우 get_current_time 함수를 여러 번 호출 (for 활용)
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-07-02 18:09:53 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1QMzd3XTnx5km4AsvKD4q1yl', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BonlPfp8cyUj1LozLNaDBlrbJf8sJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--04985ec6-dc19-4eeb-8395-06cfac573bd0-0', tool_cal

In [6]:
# 함수 실행 결과를 문장으로 출력하기
llm_with_tools.invoke(messages)

AIMessage(content='부산은 지금 2025년 7월 2일 18시 09분 53초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 192, 'total_tokens': 218, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BonlRgg3U2lgoPEEprO9LjUXxV5gO', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f44aae19-dbbd-451c-a5ce-d6eda557ff20-0', usage_metadata={'input_tokens': 192, 'output_tokens': 26, 'total_tokens': 218, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
# pydantic으로 함수 입력값 형식 표현
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")

In [9]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown()

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {
    "get_current_time": get_current_time,
    "get_yf_stock_history": get_yf_stock_history,
}

llm_with_tools = llm.bind_tools(tools)

In [10]:
messages.append(HumanMessage("테슬라는 한 달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_brK7aSBs6i7UcJk6kQYvx5cq', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 283, 'total_tokens': 310, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BonnL1V917Nf4syFN5TDJoJWnPkGg', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--d7dc7c4b-06a8-434b-b9a4-0b4e5e821362-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_brK7aSBs6i7UcJk6kQYvx5cq', 'type': 'tool_call'}] usage_metadata={'inp

In [12]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-06-02 00:00:00-04:00 | 343.5  | 348.02 | 333.33 |  342.69 | 8.18738e+07 |           0 |              0 |\n| 2025-06-03 00:00:00-04:00 | 346.6  | 355.4  | 343.04 |  344.27 | 9.93245e+07 |           0 |              0 |\n| 2025-06-04 00:00:00-04:00 | 345.1  | 345.6  | 327.33 |  332.05 | 9.89121e+07 |           0 |              0 |\n| 2025-06-05 00:00:00-04:00 | 322.49 | 324.55 | 273.21 |  284.7  | 2.875e+08   |           0 |              0 |\n| 2025-06-06 00:00:00-04:00 | 298.83 | 305.5  | 291.14 |  295.14 | 1.64748e+08 |           0 |              0 |\n| 2025-06-09 00:00:00-04:00 | 285.96 | 309.83 | 281.85 |  308.58 | 1.40909e+08 |           0 |              0 |\n| 2025-06-10 00:00:00-04:0

In [13]:
# 자연어로 함수 결과 처리하기
llm_with_tools.invoke(messages)

AIMessage(content='한 달 전, 즉 2025년 6월 1일경 테슬라(TSLA)의 주가는 약 342.69 달러였습니다. 현재 주가는 300.71 달러입니다. \n\n따라서, 테슬라의 주가는 한 달 전에 비해 약 41.98 달러 하락했습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1589, 'total_tokens': 1662, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BonrC1gTV6Gl8JDylZiok6mxQozZH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5cd6cdfe-550c-4b21-af83-8ef06ea02b6f-0', usage_metadata={'input_tokens': 1589, 'output_tokens': 73, 'total_tokens': 1662, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})